In [2]:
from python_help import *
from ortools.sat.python import cp_model


In [2]:
# def length(point1, point2):
#     return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
# def cost_result(used,solution):
#     obj = sum([f.setup_cost*used[f.index] for f in facilities])
#     for customer in customers:
#         obj += length(customer.location, facilities[solution[customer.index]].location)
#     return obj 


In [3]:
input_data = open("./data/fl_25_2").read()

facilities,customers = read_data(input_data)

In [4]:
customer_cost = np.zeros((len(customers),len(facilities)))
for c in range(len(customers)):
    for f in range(len(facilities)):
        customer_cost[c,f] = length(customers[c].location,facilities[f].location)
        
        
customer_cost = pd.DataFrame(customer_cost)
customer_damend = pd.Series(list(map(lambda x:x.demand,customers )))
facility_cost = pd.Series(list(map(lambda x:x.setup_cost,facilities)))
facility_capacity = pd.Series(list(map(lambda x:x.capacity,facilities)))

In [5]:
customer_size = len(customer_damend)
facility_size = len(facility_cost)


In [17]:
model = cp_model.CpModel()

In [18]:
fs = [
    model.NewBoolVar(name="f" + str(i))
    for i in range(facility_size)
]

In [49]:
xs = [model.NewIntVar(name=f"x{i}",lb=0,ub=facility_size-1) for i in range(customer_size)]
# xs = [model.NewIntervalVar(name=f"x{i}",start=0,end=25,size=25) for i in range(customer_size)]

In [78]:
z = model.NewBoolVar("z")
z.Not(z)


TypeError: Not() takes 1 positional argument but 2 were given

In [51]:
xs_cost = [model.NewIntVar(name=f"xc{i}",lb=0,ub=facility_size-1) for i in range(customer_size)]


In [52]:
model.AddElement(xs[0],fs,True)

In [42]:
model.AddElement(xs[0],
                 list(customer_cost.loc[0].values.astype("int")),
                 xs_cost[0])

In [61]:
a = model.NewIntervalVar(start=0,end=25,size=25,name="a")

In [62]:
a == xs[0]

TypeError: bad operand type for unary -: 'IntervalVar'

In [56]:
list(map(lambda x:(x == 0) * customer_damend[0],xs))

TypeError: unsupported operand type(s) for *: 'LinearInequality' and 'int'

In [19]:
cs = [
    model.NewBoolVar(name="c%s,%s" %(i,j))
    for i in range(customer_size)
    for j in range(facility_size)
]
cs = np.array(cs).reshape((customer_size,facility_size))
model.AddElement()

In [20]:
for i in range(customer_size):
    row = cs[i,:]
#     model.AddMaxEquality(args=row,target=1)
    model.AddSumConstraint(variables=row,lb=1,ub=1)
#     model.Add(sum(row) == 1)

In [21]:
for f,c in zip(fs * facility_capacity,cs.transpose().dot(customer_damend)):
    model.Add(c <= f)

In [22]:
objective = sum(fs * facility_cost.astype("int")) + sum(sum(cs * customer_cost.values.astype("int")))


In [23]:
model.Minimize(objective)
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20.0
status = solver.Solve(model)
solver.StatusName(status)

'FEASIBLE'

In [16]:
solution = np.argmax(pd.DataFrame(cs).applymap(solver.Value).values,axis=1)
used = list(pd.Series(fs).map(solver.Value).astype("int"))
used = np.array(used)
used[solution] = 1
print(cost_result(used=used,solution=solution,facilities=facilities,customers=customers))


4788425.624205521
